In [25]:
import gymnasium as gym
import numpy as np

In [23]:
env = gym.make("ALE/Breakout-v5")
state_size = (210, 160, 3)
action_size = env.action_space.n

In [27]:
color = np.array([210, 164, 74]).mean()

def preprocess_state(state):

    #crop and resize the image
    image = state[1:176:2, ::2]

    #convert the image to greyscale
    image = image.mean(axis=2)

    #improve image contrast
    image[image==color] = 0

    #normalize the image
    image = (image - 128) / 128 - 1
    
    #reshape the image
    image = np.expand_dims(image.reshape(88, 80, 1), axis=0)

    return image

In [31]:
class DQN:
    def __init__(self, state_size, action_size):
        
        #define the state size
        self.state_size = state_size
        
        #define the action size
        self.action_size = action_size
        
        #define the replay buffer
        self.replay_buffer = deque(maxlen=5000)
        
        #define the discount factor
        self.gamma = 0.9  
        
        #define the epsilon value
        self.epsilon = 0.8   
        
        #define the update rate at which we want to update the target network
        self.update_rate = 1000    
        
        #define the main network
        self.main_network = self.build_network()
        
        #define the target network
        self.target_network = self.build_network()
        
        #copy the weights of the main network to the target network
        self.target_network.set_weights(self.main_network.get_weights())
    
    def build_network(self):
            
            #define the model
            model = Sequential()
            
            #add the first convolutional layer
            model.add(Conv2D(32, (8, 8), strides=(4, 4), activation='relu', input_shape=self.state_size))
            
            #add the second convolutional layer
            model.add(Conv2D(64, (4, 4), strides=(2, 2), activation='relu'))
            
            #add the third convolutional layer
            model.add(Conv2D(64, (3, 3), strides=(1, 1), activation='relu'))
            
            #flatten the layers
            model.add(Flatten())
            
            #add the first dense layer
            model.add(Dense(512, activation='relu'))
            
            #add the output layer
            model.add(Dense(self.action_size, activation='linear'))
            
            #compile the model
            model.compile(loss='mse', optimizer=Adam(lr=0.001))
            
            return model